In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2
import tensorflow
import tensorflow.keras
from tensorflow.keras.layers import Embedding
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import clone_model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, Input, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
import albumentations as A

In [0]:
class ResidualUnit(tensorflow.keras.layers.Layer):
  def __init__(self, filters, model, strides=1, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = tensorflow.keras.activations.get(activation)
    self.main_layers = [
        tensorflow.keras.layers.Conv2D(filters, 3, strides=strides,
        padding="same", use_bias=False),
        tensorflow.keras.layers.BatchNormalization(),
        self.activation,
        tensorflow.keras.layers.Conv2D(filters, 3, strides=1,
        padding="same", use_bias=False),
        tensorflow.keras.layers.BatchNormalization()]

    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
        tensorflow.keras.layers.Conv2D(filters, 1, strides=strides,
        padding="same", use_bias=False),
        tensorflow.keras.layers.BatchNormalization()]

    self.model = self.call(model)

  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
    return self.activation(Z + skip_Z)




In [0]:
train_df_ = pd.read_csv('drive/My Drive/Colab Notebooks/playground/train.csv')
sample_sub_df = pd.read_csv('drive/My Drive/Colab Notebooks/playground/sample_submission.csv')

# not really useful, just for human understanding
class_map_df = pd.read_csv('drive/My Drive/Colab Notebooks/playground/class_map.csv')

In [0]:
HEIGHT = 137
WIDTH = 236


In [0]:
def resize(df, size=64, need_progress_bar=True):
    resized = {}
    resize_size=64
    angle=0
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            #Centering
            image_center = tuple(np.array(image.shape[1::-1]) / 2)
            matrix = cv2.getRotationMatrix2D(image_center, angle, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            #Scaling
            matrix = cv2.getRotationMatrix2D(image_center, 0, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            #Removing Blur
            #aug = A.GaussianBlur(p=1.0)
            #image = aug(image=image)['image']
            #Noise Removing
            #augNoise=A.MultiplicativeNoise(p=1.0)
            #image = augNoise(image=image)['image']
            #Removing Distortion
            #augDist=A.ElasticTransform(sigma=50, alpha=1, alpha_affine=10, p=1.0)
            #image = augDist(image=image)['image']
            #Brightness
            augBright=A.RandomBrightnessContrast(p=1.0)
            image = augBright(image=image)['image']
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            #image=affine_image(image)
            #image= crop_resize(image)
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #image=resize_image(image,(64,64))
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #gaussian_3 = cv2.GaussianBlur(image, (5,5), cv2.BORDER_DEFAULT) #unblur
            #image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
            #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
            #image = cv2.filter2D(image, -1, kernel)
            #ret,image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            resized[df.index[i]] = resized_roi.reshape(-1)
    else:
        for i in range(df.shape[0]):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            image_center = tuple(np.array(image.shape[1::-1]) / 2)
            matrix = cv2.getRotationMatrix2D(image_center, angle, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            matrix = cv2.getRotationMatrix2D(image_center, 0, 1.0)
            image = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR,
                            borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
            #Removing Blur
            #aug = A.GaussianBlur(p=1.0)
            #image = aug(image=image)['image']
            #Noise Removing
            #augNoise=A.MultiplicativeNoise(p=1.0)
            #image = augNoise(image=image)['image']
            #Removing Distortion
            #augDist=A.ElasticTransform(sigma=50, alpha=1, alpha_affine=10, p=1.0)
            #image = augDist(image=image)['image']
            #Brightness
            augBright=A.RandomBrightnessContrast(p=1.0)
            image = augBright(image=image)['image']
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            #image=affine_image(image)
            #image= crop_resize(image)
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #image=resize_image(image,(64,64))
            #image = cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            #gaussian_3 = cv2.GaussianBlur(image, (5,5), cv2.BORDER_DEFAULT) #unblur
            #image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
            #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
            #image = cv2.filter2D(image, -1, kernel)
            #ret,image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            resized[df.index[i]] = resized_roi.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

def get_dummies(df):
    cols = []
    for col in df:
        cols.append(pd.get_dummies(df[col].astype(str)))
    return pd.concat(cols, axis=1)

In [0]:
IMG_SIZE=64
N_CHANNELS=1

In [0]:
tensorflow.keras.backend.clear_session()
inputs = Input(shape = (IMG_SIZE, IMG_SIZE, 1))
model = Conv2D(filters=64, kernel_size=3, padding='SAME', activation='relu', input_shape=[IMG_SIZE, IMG_SIZE, 1],  use_bias=False)(inputs)
model = BatchNormalization()(model)
model = Activation("relu")(model)
model = MaxPool2D(pool_size=3, strides=2, padding="same")(model)

prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
  strides = 1 if filters == prev_filters else 2
  model = ResidualUnit(filters, model, strides=strides).model
  prev_filters = filters

model = tensorflow.keras.layers.GlobalAvgPool2D()(model)
# model = tensorflow.keras.layers.Flatten()(model)
model = Dense(1024, activation = "relu")(model)
model = Dropout(rate=0.3)(model)
dense = Dense(512, activation = "relu")(model)

head_root = tensorflow.keras.layers.Dense(168, activation = 'softmax', name = 'roots')(dense)
head_vowel = tensorflow.keras.layers.Dense(11, activation = 'softmax', name = 'vowels')(dense)
head_consonant = tensorflow.keras.layers.Dense(7, activation = 'softmax', name = 'consonants')(dense)

model = Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])


In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   576         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 64)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
learning_rate_reduction_root = ReduceLROnPlateau(monitor='roots_acc', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='vowels_acc', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='consonants_acc', 
                                            patience=3, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)


In [0]:
#decreased batch size
batch_size = 256
epochs = 30

In [0]:
class MultiOutputDataGenerator(tensorflow.keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [0]:
histories = []

In [35]:
number_of_training_files = 1

for i in range(number_of_training_files):
    train_df = pd.merge(pd.read_parquet(f'drive/My Drive/Colab Notebooks/playground/train_image_data_{i}.parquet'), train_df_, on='image_id').drop(['image_id'], axis=1)
    # Visualize few samples of current training dataset
    # fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(16, 8))
    # count=0
    # for row in ax:
    #     for col in row:
    #         col.imshow(resize(train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'], axis=1).iloc[[count]], need_progress_bar=False).values.reshape(IMG_SIZE, IMG_SIZE))
    #         count += 1
    # plt.show()
    
    X_train = train_df.drop(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'], axis=1)
    X_train = resize(X_train)/255
    
    # CNN takes images in shape `(batch_size, h, w, channels)`, so reshape the images
    X_train = X_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, N_CHANNELS)
    
    Y_train_root = pd.get_dummies(train_df['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df['vowel_diacritic']).values
    Y_train_consonant = pd.get_dummies(train_df['consonant_diacritic']).values

    print(f'Training images: {X_train.shape}')
    print(f'Training labels root: {Y_train_root.shape}')
    print(f'Training labels vowel: {Y_train_vowel.shape}')
    print(f'Training labels consonants: {Y_train_consonant.shape}')

    # Divide the data into training and validation set
    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(X_train, Y_train_root, Y_train_vowel, Y_train_consonant, test_size=0.08, random_state=666)
    del train_df
    del X_train
    del Y_train_root, Y_train_vowel, Y_train_consonant

    # Data augmentation for creating more training data
    datagen = MultiOutputDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.15, # Randomly zoom image 
        width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


    # This will just calculate parameters required to augment the given data. This won't perform any augmentations
    datagen.fit(x_train)

    # Fit the model
    history = model.fit_generator(datagen.flow(x_train, {'roots': y_train_root, 'vowels': y_train_vowel, 'consonants': y_train_consonant}, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                              steps_per_epoch=x_train.shape[0] // batch_size, 
                              callbacks=[learning_rate_reduction_root, learning_rate_reduction_vowel, learning_rate_reduction_consonant])

    histories.append(history)
    
    # Delete to reduce memory usage
    del x_train
    del x_test
    del y_train_root
    del y_test_root
    del y_train_vowel
    del y_test_vowel
    del y_train_consonant
    del y_test_consonant
    gc.collect()


Training images: (50210, 64, 64, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Epoch 1/30
180/180 [==============================] - 46s 255ms/step - loss: 0.6434 - roots_loss: 0.3581 - vowels_loss: 0.1634 - consonants_loss: 0.1220 - roots_acc: 0.9040 - vowels_acc: 0.9546 - consonants_acc: 0.9642 - val_loss: 1.2944 - val_roots_loss: 0.5152 - val_vowels_loss: 0.5129 - val_consonants_loss: 0.2663 - val_roots_acc: 0.8586 - val_vowels_acc: 0.8603 - val_consonants_acc: 0.9106
Epoch 2/30
180/180 [==============================] - 45s 251ms/step - loss: 0.5341 - roots_loss: 0.2885 - vowels_loss: 0.1459 - consonants_loss: 0.0997 - roots_acc: 0.9186 - vowels_acc: 0.9571 - consonants_acc: 0.9707 - val_loss: 0.3641 - val_roots_loss: 0.2154 - val_vowels_loss: 0.0770 - val_consonants_loss: 0.0718 - val_roots_acc: 0.9403 - val_vowels_acc: 0.9801 - val_consonants_acc: 0.9798
Epoch 3/30
180/180 [==============================] - 45s 25


Training images: (50210, 64, 64, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Epoch 1/30
180/180 [==============================] - 46s 253ms/step - loss: 0.4902 - roots_loss: 0.2748 - vowels_loss: 0.1234 - consonants_loss: 0.0920 - roots_acc: 0.9269 - vowels_acc: 0.9661 - consonants_acc: 0.9742 - val_loss: 0.3226 - val_roots_loss: 0.1969 - val_vowels_loss: 0.0613 - val_consonants_loss: 0.0644 - val_roots_acc: 0.9507 - val_vowels_acc: 0.9861 - val_consonants_acc: 0.9823
Epoch 2/30
180/180 [==============================] - 45s 251ms/step - loss: 0.4114 - roots_loss: 0.2233 - vowels_loss: 0.1104 - consonants_loss: 0.0777 - roots_acc: 0.9365 - vowels_acc: 0.9689 - consonants_acc: 0.9772 - val_loss: 0.3516 - val_roots_loss: 0.2046 - val_vowels_loss: 0.0770 - val_consonants_loss: 0.0700 - val_roots_acc: 0.9487 - val_vowels_acc: 0.9818 - val_consonants_acc: 0.9801
Epoch 3/30
180/180 [==============================] - 45s 25


Training images: (50210, 64, 64, 1)
Training labels root: (50210, 168)
Training labels vowel: (50210, 11)
Training labels consonants: (50210, 7)
Epoch 1/30
180/180 [==============================] - 46s 253ms/step - loss: 0.4485 - roots_loss: 0.2573 - vowels_loss: 0.1111 - consonants_loss: 0.0801 - roots_acc: 0.9360 - vowels_acc: 0.9692 - consonants_acc: 0.9779 - val_loss: 0.2987 - val_roots_loss: 0.1805 - val_vowels_loss: 0.0710 - val_consonants_loss: 0.0472 - val_roots_acc: 0.9517 - val_vowels_acc: 0.9833 - val_consonants_acc: 0.9885
Epoch 2/30
180/180 [==============================] - 45s 250ms/step - loss: 0.3843 - roots_loss: 0.2108 - vowels_loss: 0.1027 - consonants_loss: 0.0707 - roots_acc: 0.9421 - vowels_acc: 0.9712 - consonants_acc: 0.9801 - val_loss: 0.2802 - val_roots_loss: 0.1689 - val_vowels_loss: 0.0673 - val_consonants_loss: 0.0439 - val_roots_acc: 0.9532 - val_vowels_acc: 0.9848 - val_consonants_acc: 0.9883
Epoch 3/30
180/180 [==============================] - 45s 25

In [36]:
import os
model_name = 'ResNet_0303.h5'
save_dir = os.path.join("drive/My Drive/Colab Notebooks/playground/", 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at drive/My Drive/Colab Notebooks/playground/saved_models/ResNet_0303.h5 
